In [2]:
import polars as pl
import numpy as np
from pathlib import Path
from polimi.utils._custom import load_history, load_articles, load_behaviors

In [3]:
base_path = Path('../../dataset')
type = 'small'
split = 'train'
CSV_STORAGE = Path('/Users/lorecampa/Desktop/Projects/RecSysChallenge2024') / 'dataset' / 'graph' / type
CSV_STORAGE.mkdir(parents=True, exist_ok=True)

a = load_articles(base_path, type)
h = load_history(base_path, type, split)
b = load_behaviors(base_path, type, split)

In [5]:
h

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"
…,…,…,…,…
1479974,"[2023-05-18 06:03:16, 2023-05-18 06:03:24, … 2023-05-18 06:41:22]","[58.0, 100.0, … 21.0]","[9770989, 9769553, … 9766140]","[8.0, 124.0, … 9.0]"
2405403,"[2023-04-30 19:48:27, 2023-04-30 19:48:29, … 2023-04-30 19:49:49]","[40.0, 100.0, … 39.0]","[9743574, 9740618, … 9743487]","[1.0, 64.0, … 120.0]"
2454548,"[2023-05-17 09:35:06, 2023-05-17 09:35:23, … 2023-05-17 09:43:09]","[12.0, 10.0, … 79.0]","[9768328, 9769328, … 9634567]","[9.0, 6.0, … 11.0]"


# Fix

In [75]:
a = a.with_columns(pl.when(pl.col('category_str') == 'bibliotek').then(2504).otherwise(pl.col('category')).alias('category'))
a = a.with_columns(pl.when(pl.col('category') == 68).then(pl.lit("video")).otherwise(pl.col('category_str')).alias('category_str'))

#creation homepage node
home_page_article_dict = {k: None for k in a.columns}
home_page_article_id = 0
home_page_article_dict['article_id'] = home_page_article_id
home_page_article_dict['title'] = 'homepage'
home_page_article_dict['article_type'] = 'homepage'
a = a.vstack(pl.DataFrame(home_page_article_dict, schema=a.schema))

#link null article id in impression to homepage node
b = b.with_columns(pl.col('article_id').fill_null(home_page_article_id))


## Creation Neo4j CSV

In [76]:
b_tmp = b.select('user_id', 'gender', 'postcode', 'age').group_by('user_id').agg(pl.all().drop_nulls().first())
u_data = h.select('user_id').join(b_tmp, on='user_id', how='left').sort('user_id').unique('user_id')
u_data = u_data.with_columns(pl.lit('User').alias(':LABEL'))
u_data=u_data.rename({'user_id': 'id:ID(User)', 'gender':'gender:int', 'postcode':'postcode:int', 'age':'age:int'})

u_data.write_csv(CSV_STORAGE / 'user_data.csv')
u_data.head(3)

id:ID(User),gender:int,postcode:int,age:int,:LABEL
u32,i8,i8,i8,str
10068,null,null,null,"""User"""
10200,null,null,null,"""User"""
10201,null,null,null,"""User"""


In [77]:
topic = a.select(pl.col('topics').alias('topic')).explode('topic').unique().sort('topic').drop_nulls()
topic = topic.insert_column(0, pl.Series("topic_id", np.arange(topic.shape[0])))
topic = topic.with_columns(pl.lit('Topic').alias(':LABEL'))
topic=topic.rename({'topic': 'name', 'topic_id': 'id:ID(Topic)'})
topic.write_csv(CSV_STORAGE / 'topic_data.csv')
topic.head(3)

id:ID(Topic),name,:LABEL
i64,str,str
0,"""Ansættelsesforhold""","""Topic"""
1,"""Bandekriminalitet""","""Topic"""
2,"""Bedrageri""","""Topic"""


In [78]:
entity_group = a.select(pl.col('entity_groups').alias('entity_group')).explode('entity_group').unique().sort('entity_group').drop_nulls()
entity_group = entity_group.insert_column(0, pl.Series("entity_group_id", np.arange(entity_group.shape[0])))
entity_group = entity_group.with_columns(pl.lit('Entity').alias(':LABEL'))
entity_group = entity_group.rename({'entity_group': 'name', 'entity_group_id': 'id:ID(Entity)'})
entity_group.write_csv(CSV_STORAGE / 'entity_data.csv')
entity_group.head(3)

id:ID(Entity),name,:LABEL
i64,str,str
0,"""EVENT""","""Entity"""
1,"""LOC""","""Entity"""
2,"""MISC""","""Entity"""


In [79]:
cats = a.select(pl.col('category').alias('category_id'), pl.col('category_str').alias('name')).unique().drop_nulls()
ids_cats = cats.select('category_id').to_numpy().reshape(-1)
sub_cats = a.select(pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()
sub_cats = sub_cats.filter(~pl.col('category_id').is_in(cats.select('category_id'))).with_columns(pl.lit(None).alias('name'))
category = cats.vstack(sub_cats).sort('category_id')
category = category.with_columns(pl.lit('Category').alias(':LABEL')).sort('category_id').unique()
category=category.rename({'category_id': 'id:ID(Category)'})
category.write_csv(CSV_STORAGE / 'category_data.csv')
category.head(3)

id:ID(Category),name,:LABEL
i16,str,str
496,null,"""Category"""
268,null,"""Category"""
540,null,"""Category"""


In [80]:
a_to_category = a.select('article_id', pl.lit(False).alias('is_sub'), pl.col('category').alias('category_id')).unique().drop_nulls()
a_to_sub_category = a.select('article_id', pl.lit(True).alias('is_sub'), pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()

a_to_category = a_to_category.vstack(a_to_sub_category).rename({'article_id': ':START_ID(Article)', 'category_id': ':END_ID(Category)'})\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
    
a_to_category = a_to_category.rename({'is_sub':'is_sub:boolean'})
a_to_category.write_csv(CSV_STORAGE / 'article_to_category.csv')
a_to_category.head(3)

:START_ID(Article),is_sub:boolean,:END_ID(Category),:TYPE
i32,bool,i16,str
9625399,false,118,"""IS_IN"""
9730470,false,142,"""IS_IN"""
4894960,false,498,"""IS_IN"""


In [81]:
a_to_topic = a.select('article_id', pl.col('topics').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_topic = a_to_topic.join(topic, on='name', how='left').select('article_id', pl.col('id:ID(Topic)').alias('topic_id'))\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
a_to_topic = a_to_topic.rename({'article_id': ':START_ID(Article)', 'topic_id': ':END_ID(Topic)'})
a_to_topic.write_csv(CSV_STORAGE / 'article_to_topic.csv')
a_to_topic.head(3)

:START_ID(Article),:END_ID(Topic),:TYPE
i32,i64,str
9717707,27,"""IS_IN"""
9739202,21,"""IS_IN"""
9561345,10,"""IS_IN"""


In [82]:
a_to_entity = a.select('article_id', pl.col('entity_groups').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_entity = a_to_entity.join(entity_group, on='name', how='left').select('article_id', pl.col('id:ID(Entity)').alias('entity_id'))\
        .with_columns(pl.lit('IS_IN').alias(':TYPE'))

a_to_entity = a_to_entity.rename({'article_id': ':START_ID(Article)', 'entity_id': ':END_ID(Entity)'})
a_to_entity.write_csv(CSV_STORAGE / 'article_to_entity.csv')
a_to_entity.head(3)

:START_ID(Article),:END_ID(Entity),:TYPE
i32,i64,str
9659666,1,"""IS_IN"""
9730006,4,"""IS_IN"""
9759301,5,"""IS_IN"""


In [83]:
a_data = a.drop('image_ids', 'ner_clusters', 'entity_groups', 'topics', 'subcategory', 
                'category', 'category_str')\
                    .with_columns(pl.col('title').str.replace_all('\n', ''))\
                    .with_columns(pl.col('subtitle').str.replace_all('\n', ''))\
                    .with_columns(pl.col('body').str.replace_all('\n', ''))\
                    .with_columns(pl.lit('Article').alias(':LABEL'))\
                    .sort('article_id')

a_data = a_data.rename({'article_id': 'id:ID(Article)', 'premium':'premium:boolean', 'total_inviews': 'total_inviews:int', 
                        'total_pageviews': 'total_pageviews:int', 'sentiment_score':'sentiment_score:float', 'total_read_time': 'total_read_time:float'})
a_data.write_csv(CSV_STORAGE / 'article_data.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
a_data.head(3)

id:ID(Article),title,subtitle,last_modified_time,premium:boolean,body,published_time,article_type,url,total_inviews:int,total_pageviews:int,total_read_time,sentiment_score:float,sentiment_label,:LABEL
i32,str,str,datetime[μs],bool,str,datetime[μs],str,str,i32,i32,f32,f32,str,str
0,"""homepage""",null,null,null,null,null,"""homepage""",null,null,null,null,null,null,"""Article"""
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,"""article_default""","""https://ekstrabladet.dk/krimi/…",null,null,null,0.9955,"""Negative""","""Article"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,"""article_default""","""https://ekstrabladet.dk/underh…",null,null,null,0.846,"""Positive""","""Article"""


In [84]:
u_to_a = pl.concat([
    slice.explode(pl.all().exclude('user_id'))
    for slice in h.iter_slices(100000)
]).sort('user_id').with_columns(pl.lit('CLICK').alias(':TYPE'))
u_to_a = u_to_a.rename({'user_id': ':START_ID(User)', 'article_id_fixed': ':END_ID(Article)',
                        'impression_time_fixed': 'impression_time:string', 'scroll_percentage_fixed': 'scroll_percentage:float', 'read_time_fixed': 'read_time:float'})
u_to_a.write_csv(CSV_STORAGE / 'user_to_article.csv')
u_to_a.head(3)

:START_ID(User),impression_time:string,scroll_percentage:float,:END_ID(Article),read_time:float,:TYPE
u32,datetime[μs],f32,i32,f32,str
10068,2023-04-27 14:22:26,100.0,9735753,39.0,"""CLICK"""
10068,2023-04-27 18:51:53,100.0,9739065,21.0,"""CLICK"""
10068,2023-04-30 08:02:28,76.0,9667501,83.0,"""CLICK"""


### History

In [69]:
# batch = 0
# batch_size=100
# user_ids = h.select('user_id')

# h_final = None
# for batch in range(0, len(user_ids), batch_size):
    
#     user_ids_batch = user_ids[batch:batch+batch_size]
#     h_data = h.filter(pl.col('user_id').is_in(user_ids[batch:batch+batch_size]))\
#     .explode('impression_time_fixed', 'article_id_fixed', 'scroll_percentage_fixed', 'read_time_fixed')

#     h_data = h_data.with_columns(pl.Series(np.arange(-1, len(h_data) - 1)).cast(pl.Int32).alias('id_l'))
#     h_data = h_data.with_columns(pl.Series(np.arange(len(h_data))).cast(pl.Int32).alias('id_r'))
#     h_data = h_data.join(h_data, left_on=['user_id', 'id_r'], right_on=['user_id', 'id_l'], how='left')

#     h_data = h_data.select(
#         pl.col('article_id_fixed').alias(':START_ID(Article)'),
#         'user_id', 
#         pl.col('impression_time_fixed').alias('impression_time'),
#         pl.col('scroll_percentage_fixed').alias('scroll_percentage'),
#         pl.col('read_time_fixed').alias('read_time'),
#         pl.col('article_id_fixed_right').alias(':END_ID(Article)'),
#         pl.lit('FOLLOW').alias(':TYPE')
#     ).drop_nulls(':END_ID(Article)')
    
#     if h_final is None:
#         h_final = h_data
#     else:
#         h_final = h_final.vstack(h_data)
    

# h_final.write_csv(CSV_STORAGE / 'history_article_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
# h_final.head(3)

In [70]:
# h_data = h.select(pl.col('user_id').alias(':START_ID(User)'), 
#                   pl.col('impression_time_fixed').list.first().alias('impression_time'),
#                   pl.col('scroll_percentage_fixed').list.first().alias('scroll_percentage'),
#                   pl.col('read_time_fixed').list.first().alias('read_time'),
#                   pl.col('article_id_fixed').list.first().alias(':END_ID(Article)'),
#                   pl.lit('FOLLOW').alias(':TYPE'))

# h_data.write_csv(CSV_STORAGE / 'history_user_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
# h_data.head(3)

In [71]:
# ' '.join(
#     [
#         'bin/neo4j-admin database import full',
#         #nodes
#         f'--nodes=User=import/{type}/user_data.csv',
#         f'--nodes=Article=import/{type}/article_data.csv',
#         #relationships
#         f'--relationships=import/{type}/history_user_to_article.csv',
#         f'--relationships=import/{type}/history_article_to_article.csv',
        
#         '--overwrite-destination',
#         f'newsdb-history-{type}'
#         ])

### Load neo4j-admin-script

In [85]:
' '.join(
    [
        'bin/neo4j-admin database import full',
        #nodes
        f'--nodes=import/{type}/user_data.csv',
        f'--nodes=import/{type}/article_data.csv',
        f'--nodes=import/{type}/category_data.csv',
        f'--nodes=import/{type}/entity_data.csv',
        f'--nodes=import/{type}/topic_data.csv',
        #relationships
        f'--relationships=import/{type}/user_to_article.csv',
        f'--relationships=import/{type}/article_to_topic.csv',
        f'--relationships=import/{type}/article_to_entity.csv',
        f'--relationships=import/{type}/article_to_category.csv',
        
        '--overwrite-destination',
        f'newsdb-{type}'
        ])

'bin/neo4j-admin database import full --nodes=import/small/user_data.csv --nodes=import/small/article_data.csv --nodes=import/small/category_data.csv --nodes=import/small/entity_data.csv --nodes=import/small/topic_data.csv --relationships=import/small/user_to_article.csv --relationships=import/small/article_to_topic.csv --relationships=import/small/article_to_entity.csv --relationships=import/small/article_to_category.csv --overwrite-destination newsdb-small'

## Contraints

In [1]:
from neo4j import GraphDatabase
from polimi.utils._graph import add_constraint, execute_query
from dotenv import load_dotenv
import os

load_dotenv()
# Retrieve Neo4j connection details
URI = 'neo4j://ec2-34-240-14-242.eu-west-1.compute.amazonaws.com:7687'
USER = 'neo4j'
PASSW = 'password'

database = 'newsdb-small'

# Establish connection
driver = GraphDatabase.driver(URI, auth=(USER, PASSW), database=database)

print(driver.verify_connectivity())
add_constraint(driver, 'Article', 'id')
add_constraint(driver, 'User', 'id')
add_constraint(driver, 'Topic', 'id')
add_constraint(driver, 'Category', 'id')
add_constraint(driver, 'Entity', 'id')


driver.close()

None


Failed to read from defunct connection IPv4Address(('ec2-34-240-14-242.eu-west-1.compute.amazonaws.com', 7687)) (ResolvedIPv4Address(('34.240.14.242', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('ec2-34-240-14-242.eu-west-1.compute.amazonaws.com', 7687)) (ResolvedIPv4Address(('34.240.14.242', 7687)))

In [37]:
def list_databases(driver):
    with driver.session() as session:
        result = session.run("SHOW DATABASES")
        databases = [record["name"] for record in result]
        return databases

databases = list_databases(driver)
print("Databases:", databases)

/var/folders/_k/m_1_nsps4bx58z5_3tf9v_kc0000gn/T/ipykernel_76159/3004321709.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Databases: ['neo4j', 'newsdb-small', 'system']


In [87]:
query  = """
CALL gds.fastRP.stream('myGraph',
  {
    embeddingDimension: 10,
    randomSeed: 42
  }
)
YIELD nodeId, embedding
"""


execute_query(driver, query, debug=True)

{}


{}

In [ ]:
hist